<summary>
    <font size="5" color="gray"><b> CONVENIO CENACE-UNISON 2023-2025
 </b></font>
</summary><br>
<summary>
    <font size="4" color="orange"><b> DESARROLLO DE PROTOTIPO: METODOLOGÍA CON APRENDIZAJE PROFUNDO PARA EL PRONÓSTICO A CORTO PLAZO DE DEMANDA DE ENERGÍA CON DATOS EN ADELANTO.
 </b></font>
</summary>

---

<summary>
    <font size="6" color="gray"><b> Libreta JNB1: Ingeniería de características 
 </b></font>
</summary>
<summary>
    <font size="5" color="gray"><b> Modelado parte 1
 </b></font>
</summary>

--- 

<summary>

    <font size="4" color="lightblue"><b> MODELO ENCODER-DECODER PRIMER COMPONENTE Y CALENDARIO EN ADELANTO </b></font>
</summary>

---

OLIVIA CAROLINA GUTÚ OCAMPO

REYNA YANET HERNÁNDEZ MADA

JULIO WAISSMAN VILANOVA (Responsable del proyecto)


---

<summary>
    <font size="4" color="orange"><b> INPUTS
 </b></font>
</summary>

Archivo  **'/../MODELO_M3_DATASET_1/internal_inputs/GCRNO_complete_silver_[fecha_de_creación].csv'** creado en la libreta JNB0. 

--- 
<summary>
    <font size="4" color="orange"><b> CHECK POINTS
 </b></font>
</summary>


No hay check points en esta libreta
    

---
<summary>
    <font size="4" color="orange"><b> OUTPUT (INTERNAL INPUTS)
 </b></font>
</summary>

<summary>
    <font size="3" color="gray"><b> 'PCA' folder
 </b></font>
</summary>
   
   * **'./internal_inputs/PCA/dataset_complete_components_train.csv'** : datos de entrenamiento con la primera componente PCA agregada.
   * **'./internal_inputs/PCA/scaler_model_clima.pkl'** parámetros del escalamiento estándar aplicado a las variables de clima previo al procedimiento de pca al conjunto de entrenamiento.
   * **'./internal_inputs/PCA/pca_clima.pkl'** : parámetros del procedimiento de pca a las variables de clima del conjunto de entrenamiento.
   * **'./internal_inputs/PCA/dataset_complete_components_[test/train/val] .csv'** : datos de prueba/entrenamiento/validación con la  primera  componente PCA agregada, resultado de aplicar los parámetros correspondientes al conjunto de entrenamiento.
   
<summary>
    <font size="3" color="gray"><b> 'MINMAX' folder
 </b></font>
</summary>
    
   * **'./internal_inputs/MINMAX/scalers.pkl'** : parámetros del escalamiento al intervalo $[-1,1]$ aplicado al conjunto de entrenamiento
     
<summary>
    <font size="3" color="gray"><b> 'TENSORS' folder
 </b></font>
</summary>
 
 Tensores de entrada-salida correspondientes al conjunto de entrenamiento, validación y prueba (entradas de la libreta YOO-JNB2)
 
   * **'./internal_inputs/TENSORS/X_encoder_train.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_train.npy'** 
   * **'./internal_inputs/TENSORS/y_train.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_val.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_val.npy'** 
   * **'./internal_inputs/TENSORS/y_val.npy'** 
   * **'./internal_inputs/TENSORS/X_encoder_test.npy'** 
   * **'./internal_inputs/TENSORS/X_decoder_test.npy'** 
   * **'./internal_inputs/TENSORS/y_test.npy'** 


---
<summary>
    <font size="4" color="orange"><b> OUTPUT 
 </b></font>
</summary>

   * **'outputs/model_info.txt'** : archivo con la información relevante sobre el proceso de entrenamiento.


   * ** Esta version incluye solo las variables de temperatura y  temperatura aparente para variables meteorologicas 


## 1.   Importación de librerías y funciones

In [1]:
#importar librerías
import numpy as np
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import os

# Normalización libraries
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import pickle as pk

# librarías auxiliares
from datetime import datetime, timedelta
from datetime import date

In [2]:
#Importar función PCA para procesamiento
%run 'funciones/Y00pcafunction.ipynb'

In [3]:
current_directory = os.getcwd()
parent_directory = os.path.dirname(current_directory)

print('current', current_directory)
print('parent', parent_directory)

current /LUSTRE/home/ryhernandez/MODEL3/HOURLY/HOURLY_MODELO_M3_DATASET_/4_pc_calendario
parent /LUSTRE/home/ryhernandez/MODEL3/HOURLY/HOURLY_MODELO_M3_DATASET_


## 2.   Acceso a  datos silver 

In [4]:

#leer el archivo generado en la libreta Y00
today = date.today().strftime("%Y-%m-%d")

#Guardar nuevo dataframe con  date index
path = os.path.dirname(os.getcwd()) #leer ubicacion carpeta 

df_GCRNO_complete = pd.read_csv(path + '/internal_inputs/' + 'GCRNO_complete_silver_' + today + '.csv', parse_dates=True)
df_GCRNO_complete['Date_time'] =  pd.to_datetime(df_GCRNO_complete['Date_time'],format='%Y-%m-%d %H:%M:%S')
df_GCRNO_complete.set_index("Date_time", inplace=True)
df_GCRNO_complete = df_GCRNO_complete.asfreq('h')

# Duplicar columna Date-Timed para flexibilidad
df_GCRNO_complete['Date_timed'] = df_GCRNO_complete.index
df_GCRNO_complete.tail(2)

,Energy_Demand,Hour,Day,Month,Temperature_Ca,Perceived_Temperature_Ca,Precipitation_Ca,Humidity_Ca,Wind_Velocity_Ca,Solar_Radiation_Ca,...,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY,Date_timed
Date_time,,,,,,,,,,,,,,,,,,,,,
2023-09-05 22:00:00,5229.98,22,1,9,30.2,27.6,0.0,27.7,2.6,0.0,...,0,0,0,0,0,0,0,0,0,2023-09-05 22:00:00
2023-09-05 23:00:00,5092.62,23,1,9,29.1,26.8,0.0,31.3,1.8,0.0,...,0,0,0,0,0,0,0,0,0,2023-09-05 23:00:00


In [5]:
#df_GCRNO_complete.drop(columns =['Date_timed'], inplace=True, axis=1)
df_GCRNO_complete.columns

Index(['Energy_Demand', 'Hour', 'Day', 'Month', 'Temperature_Ca',
       'Perceived_Temperature_Ca', 'Precipitation_Ca', 'Humidity_Ca',
       'Wind_Velocity_Ca', 'Solar_Radiation_Ca', 'Cloudiness_Ca',
       'Temperature_Cul', 'Perceived_Temperature_Cul', 'Precipitation_Cul',
       'Humidity_Cul', 'Wind_Velocity_Cul', 'Solar_Radiation_Cul',
       'Cloudiness_Cul', 'Temperature_Guas', 'Perceived_Temperature_Guas',
       'Precipitation_Guas', 'Humidity_Guas', 'Wind_Velocity_Guas',
       'Solar_Radiation_Guas', 'Cloudiness_Guas', 'Temperature_Guay',
       'Perceived_Temperature_Guay', 'Precipitation_Guay', 'Humidity_Guay',
       'Wind_Velocity_Guay', 'Solar_Radiation_Guay', 'Cloudiness_Guay',
       'Temperature_Herm', 'Perceived_Temperature_Herm', 'Precipitation_Herm',
       'Humidity_Herm', 'Wind_Velocity_Herm', 'Solar_Radiation_Herm',
       'Cloudiness_Herm', 'Temperature_Maz', 'Perceived_Temperature_Maz',
       'Precipitation_Maz', 'Humidity_Maz', 'Wind_Velocity_Maz',
      

In [6]:
#An additional step is added to remove all variables not related to Temperature
#Only keeping variables related to temperature
#remove precipitation, humiidity, wind_velocity, Cloudiness,

#Remove non temperature columns
columns = df_GCRNO_complete.columns

columns_toremove = ['Precipitation', 'Humidity', 'Wind_Velocity', 'Cloudiness','Solar']

filtered_columns = [col for col in columns if not any(keyword in col for keyword in columns_toremove)]

print(filtered_columns)

['Energy_Demand', 'Hour', 'Day', 'Month', 'Temperature_Ca', 'Perceived_Temperature_Ca', 'Temperature_Cul', 'Perceived_Temperature_Cul', 'Temperature_Guas', 'Perceived_Temperature_Guas', 'Temperature_Guay', 'Perceived_Temperature_Guay', 'Temperature_Herm', 'Perceived_Temperature_Herm', 'Temperature_Maz', 'Perceived_Temperature_Maz', 'Temperature_Moch', 'Perceived_Temperature_Moch', 'Temperature_Nav', 'Perceived_Temperature_Nav', 'Temperature_Nog', 'Perceived_Temperature_Nog', 'Temperature_Obr', 'Perceived_Temperature_Obr', 'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week', 'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd', 'Before_Christmas_NY', 'Christmas_NY', 'After_Christmas_NY', 'Date_timed']


In [7]:
df_GCRNO_complete = df_GCRNO_complete.loc[:, filtered_columns]
df_GCRNO_complete


,Energy_Demand,Hour,Day,Month,Temperature_Ca,Perceived_Temperature_Ca,Temperature_Cul,Perceived_Temperature_Cul,Temperature_Guas,Perceived_Temperature_Guas,...,Tuesday_Aft_Hol,Easter_week,May_1s,May_10t,Sept_16,Nov_2nd,Before_Christmas_NY,Christmas_NY,After_Christmas_NY,Date_timed
Date_time,,,,,,,,,,,,,,,,,,,,,
2010-01-01 00:00:00,1329.00,0,4,1,11.6,10.8,16.5,17.0,17.9,16.5,...,0,0,0,0,0,0,0,1,0,2010-01-01 00:00:00
2010-01-01 01:00:00,1279.00,1,4,1,10.8,10.4,13.3,13.7,15.2,14.3,...,0,0,0,0,0,0,0,1,0,2010-01-01 01:00:00
2010-01-01 02:00:00,1234.00,2,4,1,9.5,8.0,13.0,13.4,14.7,14.1,...,0,0,0,0,0,0,0,1,0,2010-01-01 02:00:00
2010-01-01 03:00:00,1179.00,3,4,1,8.5,6.8,11.9,12.1,13.0,12.8,...,0,0,0,0,0,0,0,1,0,2010-01-01 03:00:00
2010-01-01 04:00:00,1133.00,4,4,1,7.4,5.5,11.9,12.0,12.7,12.4,...,0,0,0,0,0,0,0,1,0,2010-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-05 19:00:00,5067.90,19,1,9,35.3,32.8,32.9,32.5,31.8,33.5,...,0,0,0,0,0,0,0,0,0,2023-09-05 19:00:00
2023-09-05 20:00:00,5182.19,20,1,9,32.6,29.8,29.5,31.9,30.7,32.8,...,0,0,0,0,0,0,0,0,0,2023-09-05 20:00:00
2023-09-05 21:00:00,5264.73,21,1,9,31.0,28.3,28.9,31.7,30.4,32.7,...,0,0,0,0,0,0,0,0,0,2023-09-05 21:00:00


In [8]:
df_GCRNO_complete.columns

Index(['Energy_Demand', 'Hour', 'Day', 'Month', 'Temperature_Ca',
       'Perceived_Temperature_Ca', 'Temperature_Cul',
       'Perceived_Temperature_Cul', 'Temperature_Guas',
       'Perceived_Temperature_Guas', 'Temperature_Guay',
       'Perceived_Temperature_Guay', 'Temperature_Herm',
       'Perceived_Temperature_Herm', 'Temperature_Maz',
       'Perceived_Temperature_Maz', 'Temperature_Moch',
       'Perceived_Temperature_Moch', 'Temperature_Nav',
       'Perceived_Temperature_Nav', 'Temperature_Nog',
       'Perceived_Temperature_Nog', 'Temperature_Obr',
       'Perceived_Temperature_Obr', 'Monday_Holiday', 'Tuesday_Aft_Hol',
       'Easter_week', 'May_1s', 'May_10t', 'Sept_16', 'Nov_2nd',
       'Before_Christmas_NY', 'Christmas_NY', 'After_Christmas_NY',
       'Date_timed'],
      dtype='object')

## 3.   Creación de conjuntos de entrenamiento, validación y prueba

In [9]:
#to adjust the set
#partial = pd.Timestamp("2023-01-31")
#print(partial)
#df_GCRNO_complete = df_GCRNO_complete[df_GCRNO_complete['Date_timed'] <= partial]
#df_GCRNO_complete.tail(5)


In [10]:
# Definir las fechas de división para los conjuntos de validación y prueba
# **Testing:** Conjunto de Prueba toma en cuenta los ultimos 12 meses
# **Validation** Validación toma en cuenta  4 meses previos al período de prueba
# **Training:** Conjunto de entrenamiento, toma en cuenta el resto de los datos después de dividir  prueba y validación
testing_start = df_GCRNO_complete.index.max() - relativedelta(months = 1) ## PARA LA VERSIÓN FINAL CAMBIAR 1 POR 12
validation_start = testing_start - relativedelta(months = 1)  ## PARA LA VERSIÓN FINAL CAMBIAR 1 POR 4

In [11]:
# Dividir los datos entrenamiento, validación y prueba
training_data = df_GCRNO_complete.loc[df_GCRNO_complete.index <= validation_start]
validation_data = df_GCRNO_complete.loc[(df_GCRNO_complete.index > validation_start) & (df_GCRNO_complete.index <= testing_start)]
testing_data = df_GCRNO_complete.loc[df_GCRNO_complete.index >testing_start]

In [12]:
#Exportar información de set de entrenamiento, validación y prueba al archivo model_info.txt

In [13]:
%%capture cap --no-stderr
print(f'Training data size: {len(training_data)}')
print(f'Validation data size: {len(validation_data)}')
print(f'Testing data size: {len(testing_data)}')

In [14]:
with open('./outputs/model_info.txt', 'w') as f:
    f.write(cap.stdout)

In [15]:
%%capture cap --no-stderr
print('Training set:', training_data.index.min(),'-TO-', training_data.index.max())
print('Validation set:', validation_data.index.min(),'-TO-', validation_data.index.max())
print('Testing set:', testing_data.index.min(),'-TO-',testing_data.index.max())

In [16]:
with open('./outputs/model_info.txt', 'a') as f:
    f.write(cap.stdout)

## 4.    PCA 

In [17]:
# Nuevas columnas en set de entrenamiento
## Se aplica análisis de componentes principales a variables climáticas, después de la estandarización de las mismas
## Se guardan los parámetros de  estandarización y PCA n
## Se ejecuta función 'exogen_pca' :
### 'dataset_complete_components_train' : datos de entrenamiento con primeras 2 componentes de PCA añadidas
### 'pca_clima' :parámetros de entrenamiento de procedimiento PCA
### 'scaler_model_clima' : parámetros de entrenamiento de std scaler
dataset_complete_components_train, pca_clima, scaler_model_clima = exogen_pca(training_data, 'Date_timed')
dataset_complete_components_train.to_csv('./internal_inputs/PCA/dataset_complete_components_train.csv')

## Exportando parámetros de procesamiento para su uso fuera de la libreta
pk.dump(pca_clima, open('./internal_inputs/PCA/pca_clima.pkl','wb'))
pk.dump(scaler_model_clima, open('./internal_inputs/PCA/scaler_model_clima.pkl','wb'))

# Aplicar parámetros de training a
## set de validación
dataset_complete_components_val = transform_data(validation_data,'Date_timed',pca_clima,scaler_model_clima)
dataset_complete_components_val.to_csv('./internal_inputs/PCA/dataset_complete_components_val.csv')
## set de prueba
dataset_complete_components_test = transform_data(testing_data,'Date_timed',pca_clima,scaler_model_clima)
dataset_complete_components_test.to_csv('./internal_inputs/PCA/dataset_complete_components_test.csv')

## 5.    MINMAX [-1,1]

In [18]:
Features =[ 'Energy_Demand', 'Day', 'Hour', 'Month','PC1_Weather',
            'Monday_Holiday', 'Tuesday_Aft_Hol', 'Easter_week', 'May_1s', 'May_10t', 
            'Sept_16', 'Nov_2nd', 'Before_Christmas_NY', 'Christmas_NY', 'After_Christmas_NY']

In [19]:
# Reescalar set de entrenamiento
train = dataset_complete_components_train
scalers = {} #dicccionario

for attr in Features:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    s_s = scaler.fit_transform(train[attr].values.reshape(-1,1))
    scalers[attr] = scaler
    train[attr] = s_s.ravel()

#   Exportar parámetros
pk.dump(scalers, open('./internal_inputs/MINMAX/scalers.pkl','wb'))    
    
# Applicando parámetros de training a
## set de validación
val = dataset_complete_components_val
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(val[attr].values.reshape(-1,1))
    val[attr] = s_s.ravel()
## Test set
test = dataset_complete_components_test
for attr in Features:
    scaler = scalers[attr]
    s_s = scaler.transform(test[attr].values.reshape(-1,1))
    test[attr] = s_s.ravel()

## 6. Generación de tensores

In [20]:
#Ajustando hiperparámetros
n_pasado = (24 * 7) + 12
n_futuro = 36

In [21]:

def divide_series(series, n_pasado, n_futuro):
    """
    n_pasado: number of past observations for the encoder
    n_futuro: number of future observations and inputs given in advance for the decoder
    """
    X_encoder, X_decoder, y = list(), list(), list()  
    generador = range(len(series))

    for ini in generador:
        fin_anterior = ini + n_pasado
        fin_actual = fin_anterior + n_futuro
        if fin_actual > len(series):
            break
        past_encoder = series[ini: fin_anterior, :]
        future_decoder = series[fin_anterior: fin_actual, 4:15]
        future_output = series[fin_anterior: fin_actual, :1]
        X_encoder.append(past_encoder)
        X_decoder.append(future_decoder)
        y.append(future_output)
             
    return np.array(X_encoder), np.array(X_decoder), np.array(y)


In [22]:
all_features = ['Energy_Demand', 
                'Day', 
                'Hour', 
                'Month', 
                'PC1_Weather',
                'Monday_Holiday', 
                'Tuesday_Aft_Hol', 
                'Easter_week',
                'May_1s', 
                'May_10t', 
                'Sept_16', 
                'Nov_2nd', 
                'Before_Christmas_NY',
                'Christmas_NY', 
                'After_Christmas_NY'
               ]

In [23]:
#Definir tensores
X_encoder_train, X_decoder_train, y_train = divide_series(train[all_features].values, n_pasado, n_futuro)
X_encoder_val, X_decoder_val, y_val = divide_series(val[all_features].values, n_pasado, n_futuro)
X_encoder_test, X_decoder_test, y_test = divide_series(test[all_features].values, n_pasado, n_futuro) 

## 7. Guardado de tensores

In [24]:
%%capture cap --no-stderr
print('X_encoder_train tensor size:', X_encoder_train.shape,'\n', 
      'X_decoder_train tensor size:', X_decoder_train.shape,'\n',
      'y_train tensor size:', y_train.shape
     )
print('X_encoder_val tensor size:', X_encoder_val.shape,'\n', 
      'X_decoder_val tensor size:', X_decoder_val.shape,'\n',
      'y_val tensor size:', y_val.shape
     )
print('X_encoder_test tensor size:', X_encoder_test.shape,'\n', 
      'X_decoder_test tensor size:', X_decoder_test.shape,'\n',
      'y_test tensor size:', y_test.shape
     )
print('DATE:', today,'\n'
     )

In [25]:
with open('./outputs/model_info.txt', 'a') as f:
    f.write(cap.stdout)

In [26]:
np.save('./internal_inputs/TENSORS/X_encoder_train.npy', X_encoder_train)
np.save('./internal_inputs/TENSORS/X_decoder_train.npy', X_decoder_train)
np.save('./internal_inputs/TENSORS/y_train.npy', y_train)
np.save('./internal_inputs/TENSORS/X_encoder_val.npy', X_encoder_val)
np.save('./internal_inputs/TENSORS/X_decoder_val.npy', X_decoder_val)
np.save('./internal_inputs/TENSORS/y_val.npy', y_val)
np.save('./internal_inputs/TENSORS/X_encoder_test.npy', X_encoder_test)
np.save('./internal_inputs/TENSORS/X_decoder_test.npy', X_decoder_test)
np.save('./internal_inputs/TENSORS/y_test.npy', y_test)

<summary>
    <font size="4" color="gray"> Maestría en Ciencia de Datos | Universidad de Sonora </font>
</summary>
<font size="1" color="gray"> Blvd. Luis Encinas y Rosales s/n Col. Centro. Edificio 3K1 planta baja C.P. 83000, Hermosillo, Sonora, México </font>
<font size="1" color="gray"> mcd@unison.mx </font>
<font size="1" color="gray"> Tel: +52 (662) 259 2155  </font>